### This notebook was not finished due to questionable ground truth of the entire gittbles dataset, as mentioned in the paper

In [1]:
from datetime import datetime
from os.path import join
from os import listdir
from os import makedirs
import json
from re import sub #for camel case conversion

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split

from sherlock import helpers
from sherlock.deploy.model import SherlockModel
# from sherlock.functional import extract_features_to_csv
from sherlock.features.paragraph_vectors import initialise_pretrained_model, initialise_nltk
from sherlock.features.preprocessing import (
    extract_features,
#     convert_string_lists_to_lists,
    prepare_feature_extraction,
#     load_parquet_values,
)
from sherlock.features.word_embeddings import initialise_word_embeddings

import altair as alt
alt.renderers.enable('default')

RendererRegistry.enable('default')

In [2]:
def camel_case(s):
    try:
      c = sub(r"(_|-)+", " ", s).title().replace(" ", "")
      return ''.join([c[0].lower(), c[1:]])
    except:
#         print(f'Error converting {s} to camel case')
        return ''

In [3]:
# !! Make sure you downloaded a gittables dataset (f.e. abstraction_tables)
# and unzipped the data into the directory mentioned below !!
path = '../data/data/gittables/whole_tables'
print(camel_case("birth Date"))
types = np.load(f"../model_files/classes_sherlock.npy", allow_pickle=True)
types = list(map(camel_case, types))
print(types)

filepaths = [join(path, f) for f in listdir(path) if f.endswith('.parquet')]

birthDate
['address', 'affiliate', 'affiliation', 'age', 'album', 'area', 'artist', 'birthDate', 'birthPlace', 'brand', 'capacity', 'category', 'city', 'class', 'classification', 'club', 'code', 'collection', 'command', 'company', 'component', 'continent', 'country', 'county', 'creator', 'credit', 'currency', 'day', 'depth', 'description', 'director', 'duration', 'education', 'elevation', 'family', 'fileSize', 'format', 'gender', 'genre', 'grades', 'industry', 'isbn', 'jockey', 'language', 'location', 'manufacturer', 'name', 'nationality', 'notes', 'operator', 'order', 'organisation', 'origin', 'owner', 'person', 'plays', 'position', 'product', 'publisher', 'range', 'rank', 'ranking', 'region', 'religion', 'requirement', 'result', 'sales', 'service', 'sex', 'species', 'state', 'status', 'symbol', 'team', 'teamName', 'type', 'weight', 'year']


In [4]:
#TODO RERUN, Types where wrongly formatted!!
path_out = "../data/data/gittables_benchmark/new_benchmark"
makedirs(path_out, exist_ok=True)
tables = []
table_true_types = []
col_ids = []
curr_id = 0
start = datetime.now()
print(f'Started at {start}')

for fp in filepaths:
    try:
        table = pd.read_parquet(fp)
        table.rename(columns=camel_case,inplace=True)
    except:
        print(f'Error converting table in {fp}')
        continue
    table_id = fp[fp.rfind('/')+1:-len(".parquet")]
    table_tt_list = table.columns.tolist()
    table_sherlock_types = list(filter(lambda x: x in types,table_tt_list)) #TODO: make sure column contains values
    if len(table_sherlock_types)>=3 and not table.empty:
#         print(table_id)
#         print(table[table_sherlock_types])
        print(table[table_sherlock_types].columns.tolist())
        table[table_sherlock_types].to_csv(path_out+'/'+table_id+'.csv')
#         df = pd.DataFrame(zip(*columns), columns=col_true_types)
#     for idx, col_name in enumerate(table.columns):
#         if len(col_name) == 0: #no type defined, skip col
#             continue
#         if col_name not in types:
#             continue
#         col = table.iloc[:, idx]
#         if col.count() == 0: # 0.35952832125142853
#                 continue
#         try: #TODO fix 2 columns in table with same name
#             columns.append(list(map(str, col.to_list())))
#             col_true_types.append(col_name)
#             col_ids.append(curr_id)
#             curr_id+=1
#         except:
#             print(f'error in table {fp}')
        
        
print(f'Finished loading tables at {datetime.now()}, took {datetime.now() - start} seconds')

Started at 2022-04-04 23:08:20.518893
['category', 'name', 'description']
['product', 'component', 'status']
['country', 'name', 'location', 'description']
['year', 'country', 'city']
['name', 'state', 'region']
['product', 'component', 'status']
['category', 'name', 'description', 'format']
['rank', 'description', 'address', 'city']
['artist', 'gender', 'location', 'status']
['name', 'duration', 'category']
['name', 'year', 'publisher', 'genre', 'description']
['album', 'year', 'artist']
['name', 'notes', 'organisation']
['year', 'country', 'city']
['category', 'name', 'description', 'format']
['name', 'type', 'weight']
['address', 'category', 'city', 'name', 'state']
['status', 'city', 'state', 'county']
['country', 'name', 'location', 'description']
['name', 'type', 'weight']
['year', 'country', 'city']
['name', 'state', 'region']
['name', 'type', 'notes']
['year', 'country', 'city']
['status', 'city', 'state', 'county']
['name', 'artist', 'genre']
['name', 'year', 'family']
['artis

['artist', 'gender', 'location', 'status']
['name', 'address', 'city', 'state']
['artist', 'gender', 'location', 'status']
['artist', 'gender', 'location', 'status']
['category', 'name', 'description', 'format']
['category', 'name', 'description']
['gender', 'year', 'notes']
['name', 'state', 'region']
['category', 'name', 'description']
['position', 'artist', 'genre']
['rank', 'artist', 'album']
['address', 'description', 'type']
['name', 'type', 'description']
['album', 'year', 'artist']
['name', 'artist', 'type']
['artist', 'gender', 'location', 'status']
['status', 'city', 'state', 'county']
['category', 'name', 'description', 'format']
['type', 'status', 'owner']
['status', 'city', 'state', 'county']
['status', 'city', 'state', 'county']
['gender', 'education', 'age']
['status', 'city', 'state', 'county']
['status', 'city', 'state', 'county']
['city', 'state', 'name', 'address']
['city', 'state', 'duration']
['address', 'description', 'type']
['year', 'country', 'city']
['address'

['name', 'type', 'description', 'status']
['year', 'country', 'city']
['symbol', 'city', 'state']
['artist', 'gender', 'location', 'status']
['category', 'name', 'description', 'format']
['name', 'city', 'state']
['rank', 'artist', 'duration']
['year', 'genre', 'publisher']
['status', 'city', 'state', 'county']
['year', 'country', 'city']
['status', 'city', 'state', 'county']
['category', 'name', 'description']
['type', 'year', 'description']
['category', 'name', 'description', 'format']
['year', 'country', 'city']
['category', 'address', 'city', 'state']
['name', 'type', 'description']
['symbol', 'address', 'state']
['name', 'description', 'category']
['category', 'name', 'description']
['name', 'description', 'capacity']
['name', 'description', 'duration', 'category', 'status']
['category', 'name', 'description']
['name', 'description', 'format']
['artist', 'gender', 'location', 'status']
['artist', 'gender', 'location', 'status']
['category', 'name', 'description']
['year', 'publish

['category', 'name', 'description']
['name', 'type', 'description']
['name', 'state', 'address']
['code', 'name', 'description']
['name', 'type', 'description']
['category', 'name', 'description']
['name', 'company', 'state', 'city', 'age', 'location']
['depth', 'species', 'year', 'location']
['status', 'classification', 'name', 'description', 'notes']
['status', 'city', 'state', 'county']
['artist', 'gender', 'location', 'status']
['category', 'name', 'description']
['type', 'name', 'duration']
['artist', 'gender', 'location', 'status']
['gender', 'duration', 'address']
['language', 'publisher', 'notes']
['region', 'county', 'address']
['artist', 'gender', 'location', 'status']
['status', 'creator', 'description']
['area', 'year', 'ranking']
['city', 'status', 'location']
['album', 'year', 'artist']
['address', 'description', 'type']
['category', 'name', 'description', 'format']
['category', 'name', 'description', 'format']
['name', 'type', 'format']
['rank', 'name', 'city', 'state']


['name', 'type', 'weight']
['type', 'status', 'owner']
['artist', 'gender', 'location', 'status']
['name', 'description', 'duration', 'category', 'status']
['rank', 'artist', 'duration']
['artist', 'gender', 'location', 'status']
['artist', 'gender', 'location', 'status']
['status', 'city', 'state', 'county']
['name', 'type', 'description', 'status']
['artist', 'gender', 'location', 'status']
['artist', 'gender', 'location', 'status']
['artist', 'gender', 'location', 'status']
['artist', 'gender', 'location', 'status']
['status', 'city', 'state', 'county']
['name', 'organisation', 'notes']
['status', 'type', 'code', 'notes']
['address', 'category', 'collection', 'company', 'country', 'description', 'name', 'notes', 'organisation', 'region', 'status']
['component', 'type', 'owner', 'status', 'description']
['artist', 'gender', 'location', 'status']
['artist', 'gender', 'location', 'status']
['name', 'address', 'type']
['artist', 'gender', 'location', 'status']
['sex', 'name', 'category'

...

['name', 'address', 'type']
['name', 'state', 'region']
['album', 'artist', 'name']
['location', 'category', 'name']
['product', 'currency', 'location']
['gender', 'duration', 'address']
['duration', 'language', 'country']
['city', 'area', 'age', 'collection']
['year', 'country', 'city']
['status', 'creator', 'description']
['product', 'component', 'status']
['year', 'country', 'city']
['jockey', 'age', 'owner', 'class']
['status', 'city', 'state', 'county']
['category', 'name', 'description', 'format']
['description', 'location', 'notes']
['rank', 'artist', 'duration']
['type', 'status', 'owner']
['gender', 'year', 'notes']
['artist', 'gender', 'location', 'status']
['year', 'country', 'city']
['name', 'country', 'region', 'description']
['gender', 'age', 'country', 'state', 'day']
['gender', 'year', 'notes']
['rank', 'name', 'artist']
['language', 'notes', 'classification']
['name', 'category', 'currency', 'state', 'country']
['symbol', 'city', 'state']
['name', 'category', 'descript

['status', 'city', 'state', 'county']
['category', 'name', 'description']
['name', 'type', 'description']
['name', 'year', 'family']
['product', 'component', 'status']
['status', 'type', 'description']
['artist', 'gender', 'location', 'status']
['name', 'artist', 'type']
['album', 'year', 'artist']
['artist', 'gender', 'location', 'status']
['artist', 'gender', 'location', 'status']
['area', 'year', 'ranking']
['publisher', 'language', 'type']
['album', 'year', 'artist']
['collection', 'location', 'person', 'position', 'state']
['name', 'species', 'description', 'notes']
['name', 'description', 'format']
['publisher', 'language', 'type']
['year', 'country', 'city']
['area', 'type', 'description']
['artist', 'gender', 'location', 'status']
['address', 'description', 'type']
['address', 'description', 'type']
['category', 'name', 'description', 'format']
['artist', 'gender', 'location', 'status']
['year', 'country', 'city']
['year', 'publisher', 'type', 'age']
['name', 'description', 'fo

['product', 'component', 'status']
['name', 'city', 'state', 'address']
['status', 'description', 'region']
['name', 'state', 'region']
['creator', 'location', 'description', 'order', 'collection']
['age', 'location', 'country', 'category', 'description']
['year', 'country', 'city']
['brand', 'city', 'state', 'location']
['category', 'name', 'description', 'format']
['year', 'name', 'area']
['code', 'name', 'description']
['artist', 'gender', 'location', 'status']
['artist', 'gender', 'location', 'status']
['type', 'name', 'category']
['rank', 'artist', 'duration']
['city', 'state', 'artist']
['name', 'type', 'industry', 'description']
['year', 'publisher', 'type', 'age']
['name', 'state', 'region']
['name', 'company', 'position', 'year']
['status', 'city', 'state', 'county']
['year', 'country', 'city']
['address', 'city', 'county', 'name', 'type']
['category', 'notes', 'code']
['name', 'region', 'notes', 'area']
['state', 'city', 'artist']
['product', 'component', 'status']
['category

['artist', 'gender', 'location', 'status']
['artist', 'gender', 'location', 'status']
['category', 'company', 'location']
['name', 'address', 'type']
['gender', 'duration', 'address']
['category', 'name', 'description']
['gender', 'year', 'notes']
['name', 'address', 'type']
['name', 'notes', 'organisation']
['publisher', 'language', 'type']
['year', 'country', 'city']
['status', 'city', 'state', 'county']
['name', 'description', 'capacity']
['status', 'city', 'state', 'county']
['name', 'description', 'order', 'weight']
['type', 'name', 'address', 'city', 'county']
['album', 'year', 'artist']
['category', 'name', 'description', 'format']
['name', 'notes', 'organisation']
['isbn', 'publisher', 'description']
['artist', 'gender', 'location', 'status']
['artist', 'gender', 'location', 'status']
['status', 'city', 'state', 'county']
['category', 'name', 'description', 'format']
['code', 'name', 'description']
['artist', 'gender', 'location', 'status']
['artist', 'gender', 'location', 'sta

['service', 'brand', 'company']
['artist', 'gender', 'location', 'status']
['category', 'name', 'description']
['artist', 'gender', 'location', 'status']
['name', 'state', 'region']
['category', 'name', 'description', 'format']
['state', 'state', 'state', 'state', 'region', 'area', 'elevation', 'name', 'city']
['location', 'year', 'day']
['name', 'category', 'description']
['artist', 'gender', 'location', 'status']
['artist', 'gender', 'location', 'status']
['city', 'state', 'year']
['artist', 'gender', 'location', 'status']
['name', 'address', 'type']
['year', 'country', 'city']
['artist', 'gender', 'location', 'status']
['location', 'category', 'name']
['name', 'year', 'family']
['name', 'album', 'artist']
['gender', 'year', 'notes']
['company', 'service', 'person', 'symbol', 'family', 'year', 'code', 'age', 'product', 'country', 'day', 'result', 'state', 'owner', 'city', 'requirement', 'rank', 'location', 'type', 'order', 'brand', 'depth', 'language', 'collection', 'area', 'operator

In [5]:
# t = pd.read_parquet("../data/data/gittables/whole_tables/5755b20ac79f86bf77e92cc87cc0c82c.parquet")

In [6]:
# print(t)